In [3]:
import pandas as pd
import numpy as np
from numpy import log, dot, e, ndim
from numpy.random import rand
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.metrics import confusion_matrix
#normalizing dataframe
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F


from wrapper import t_list

2022-03-08 20:17:39.320978: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-08 20:17:39.321003: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
#parkinsons: https://www.kaggle.com/paolocons/parkinson-desease-basic-classification

In [5]:
data = pd.DataFrame()
data= pd.read_csv('./parkinsons.csv')
data

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


In [6]:
y = data["status"]
X = data.drop(["name","status"], axis=1)

In [11]:
scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)
y = y.to_numpy()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train)
y_train = F.one_hot(y_train).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test)
y_test = F.one_hot(y_test).type(torch.FloatTensor)


In [13]:
X_train[0]

tensor([0.3489, 0.1283, 0.4423, 0.0924, 0.0909, 0.0877, 0.0879, 0.0877, 0.2954,
        0.2449, 0.3713, 0.2647, 0.1786, 0.3714, 0.0589, 0.4202, 0.4617, 0.6383,
        0.3639, 0.1836, 0.4097, 0.2940])

In [8]:
# shape None == allow dynamic number of rows; X_train.shape[1] == number of features
X = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]), name='inputs')
y = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='targets')

In [9]:
# First hidden layer
# X == features coming in, 30 == number of features == neurons
h1 = tf.layers.dense(X, 20, name='dense1')
h1 = h1 * h1
# Second hidden layer
# h1 == features coming in, 26 == number of neurons == arbitrary
h2 = tf.layers.dense(h1, 10, name='dense2')
h2 = h2 * h2
# Last/output layar always has 1 neuron for binary classification problems
model1 = tf.layers.dense(h2, 2, name='dense3')

model = tf.sigmoid(model1, name='sigmoid')
# model = approx_sigmoid(model)

/tmp/ipykernel_34233/1039383836.py:3: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h1 = tf.layers.dense(X, 20, name='dense1')
/home/thuy/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/tmp/ipykernel_34233/1039383836.py:7: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  h2 = tf.layers.dense(h1, 10, name='dense2')
/tmp/ipykernel_34233/1039383836.py:10: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  model1 = tf.layers.dense(h2, 2, name='dense3')


In [10]:
loss = tf.losses.log_loss(y, model)

# Define training operation
training_op = tf.train.AdamOptimizer(.01).minimize(loss)

In [11]:
# Initialize
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    # Train model with 1000 epochs
    # saver.restore(sess, '/content/drive/MyDrive/Colab Notebooks/result/nn_data/net-1000')
    for epoch in range(1000):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        if (epoch % 100 == 0):
            training_loss = sess.run(loss, feed_dict={X: X_train, y: y_train})
            test_loss = sess.run(loss, feed_dict={X: X_test, y: y_test})
            print('epoch:',epoch,' | training loss:', training_loss, ' | test loss:', test_loss)
    saved_path = saver.save(sess, './nn_data/net', global_step=1000)

    # # Use model to classify test data
    # classifications_on_test_data = sess.run(model, feed_dict={X: X_test})
    
    # # Check model accuracy
    # classes = (classifications_on_test_data > .5).astype(int)
    
    # print('\nAccuracy Score:',metrics.accuracy_score(y_test, classes))
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({X: X_test, y: y_test}))
    no_sig_predictions = tf.equal(tf.argmax(model1, 1), tf.argmax(y, 1))
    no_sig_accuracy = tf.reduce_mean(tf.cast(no_sig_predictions, "float"))
    print('Accuracy without sigmoid: ', no_sig_accuracy.eval({X: X_test, y: y_test}))

2022-03-08 20:11:09.706779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-08 20:11:09.706805: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-08 20:11:09.706822: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thuy-ThinkPad-T480): /proc/driver/nvidia/version does not exist
2022-03-08 20:11:09.706990: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 0  | training loss: 0.680278  | test loss: 0.68641764
epoch: 100  | training loss: 0.08343455  | test loss: 0.3056145
epoch: 200  | training loss: 0.001975019  | test loss: 0.48794875
epoch: 300  | training loss: 0.0005002758  | test loss: 0.58730185
epoch: 400  | training loss: 0.0002284705  | test loss: 0.63609976
epoch: 500  | training loss: 0.0001308683  | test loss: 0.66095704
epoch: 600  | training loss: 8.469362e-05  | test loss: 0.67861295
epoch: 700  | training loss: 5.916976e-05  | test loss: 0.6921353
epoch: 800  | training loss: 4.3567954e-05  | test loss: 0.7032293
epoch: 900  | training loss: 3.3333337e-05  | test loss: 0.7127024
Accuracy: 0.9183673
Accuracy without sigmoid:  0.9183673


# PRE ENCODE

In [12]:
# Restore session
with tf.Session() as sess:
	saver.restore(sess, './nn_data/net-1000')
	with tf.variable_scope('dense1', reuse=True):
		dense1_kernel_ = sess.run(tf.get_variable('kernel'))
		dense1_bias_ = sess.run(tf.get_variable('bias'))
	with tf.variable_scope('dense2', reuse=True):
		dense2_kernel_ = sess.run(tf.get_variable('kernel'))
		dense2_bias_ = sess.run(tf.get_variable('bias'))
	with tf.variable_scope('dense3', reuse=True):
		dense3_kernel_ = sess.run(tf.get_variable('kernel'))
		dense3_bias_ = sess.run(tf.get_variable('bias'))
	encoded_input_ = X.eval({X: X_test})


INFO:tensorflow:Restoring parameters from ./nn_data/net-1000


In [13]:
print(dense1_kernel_.shape)
print(dense1_bias_.shape)
print(dense2_kernel_.shape)
print(dense2_bias_.shape)
print(dense3_kernel_.shape)
print(dense3_bias_.shape)
print(encoded_input_.shape)

(22, 20)
(20,)
(20, 10)
(10,)
(10, 2)
(2,)
(49, 22)


In [14]:
# Declare the outputs
input_size = encoded_input_.shape[0]
encoded_input = np.empty(shape=(input_size,encoded_input_.shape[1],5), dtype=np.uint64)
dense1_kernel = np.empty(shape=(22,20,5), dtype=np.uint64)
dense1_bias = np.empty(shape=(20,5), dtype=np.uint64)
dense2_kernel = np.empty(shape=(20,10,5), dtype=np.uint64)
dense2_bias = np.empty(shape=(10,5), dtype=np.uint64)
dense3_kernel = np.empty(shape=(10,2,5), dtype=np.uint64)
dense3_bias = np.empty(shape=(2,5), dtype=np.uint64)

In [15]:
# WEIGHTS
print("Weights processing...")
precision = 10
for i in range(22):
	for j in range(20):
		value = round(dense1_kernel_[i, j].item()*precision)
		for t in range(5):
			dense1_kernel[i, j, t] = value % t_list[t]

for i in range(20):
	for j in range(10):
		value = round(dense2_kernel_[i, j].item()*precision)
		for t in range(5):
			dense2_kernel[i, j, t] = value % t_list[t]

for i in range(10):
	for j in range(2):
		value = round(dense3_kernel_[i, j].item()*precision)
		for t in range(5):
			dense3_kernel[i, j, t] = value % t_list[t]

for i in range(20):
	value = round(dense1_bias_[i].item()*(precision**2))
	for t in range(5):
		dense1_bias[i, t] = value % t_list[t]

for i in range(10):
	value = round(dense2_bias_[i].item()*(precision**5))
	for t in range(5):
		dense2_bias[i, t] = value % t_list[t]

for i in range(2):
	value = round(dense3_bias_[i].item()*(precision**11))
	for t in range(5):
		dense3_bias[i, t] = value % t_list[t]

print("Done.")

Weights processing...
Done.


In [16]:
# INPUT
print("Input processing...")
for i in range(input_size):
	for j in range(encoded_input_.shape[1]):
			value = round(encoded_input_[i,j].item()*precision)
			for t in range(5):
				encoded_input[i,j,t] = value % t_list[t]

Input processing...


In [17]:
# Store the encoded tensors
np.save("./output_data/plain_layer_0", encoded_input)
np.save("./nn_data/dense1_kernel", dense1_kernel)
np.save("./nn_data/dense1_bias", dense1_bias)
np.save("./nn_data/dense2_kernel", dense2_kernel)
np.save("./nn_data/dense2_bias", dense2_bias)
np.save("./nn_data/dense3_kernel", dense3_kernel)
np.save("./nn_data/dense3_bias", dense3_bias)

print("Tensors stored")

Tensors stored


# INFERE PLAIN

In [18]:
import numpy as np
from wrapper import t_list

t_size = len(t_list)

In [19]:
# WEIGHTS
dense1_kernel = np.load("./nn_data/dense1_kernel.npy")
dense1_bias = np.load("./nn_data/dense1_bias.npy")
dense2_kernel = np.load("./nn_data/dense2_kernel.npy")
dense2_bias = np.load("./nn_data/dense2_bias.npy")
dense3_kernel = np.load("./nn_data/dense3_kernel.npy")
dense3_bias = np.load("./nn_data/dense3_bias.npy")

In [20]:
plain_input = np.load("./output_data/plain_layer_0.npy")
examples_count = plain_input.shape[0]
plain_output = np.empty((examples_count,22,5), dtype=np.uint64)

In [21]:
plain_input[...,0].dot(dense1_kernel[...,0]).shape

(49, 20)

In [22]:
# LAYER 1: dense1
print("Computing layer 1/5...")
temp = np.empty((examples_count,20,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_input[...,t_index].dot(dense1_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense1_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_1", plain_output)

Computing layer 1/5...


In [23]:
# LAYER 2: square activation function
print("Computing layer 2/5...")
plain_output = plain_output*plain_output
for t_index in range(t_size):
	plain_output[...,t_index] = plain_output[...,t_index] % t_list[t_index]
np.save("./output_data/plain_layer_2", plain_output)

Computing layer 2/5...


In [24]:
# LAYER 3: dense2
print("Computing layer 3/5...")
temp = np.empty((examples_count,10,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_output[...,t_index].dot(dense2_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense2_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_3", plain_output)

Computing layer 3/5...


In [25]:
# LAYER 4: square activation function
print("Computing layer 4/5...")
plain_output = plain_output*plain_output
for t_index in range(t_size):
	plain_output[...,t_index] = plain_output[...,t_index] % t_list[t_index]
np.save("./output_data/plain_layer_4", plain_output)

Computing layer 4/5...


In [26]:
# LAYER 5: dense3
print("Computing layer 5/5...")
temp = np.empty((examples_count,2,t_size), dtype=np.uint64)
for t_index in range(t_size):
	temp[...,t_index] = (plain_output[...,t_index].dot(dense3_kernel[...,t_index])) % t_list[t_index]
	temp[...,t_index] = (temp[...,t_index] + dense3_bias[...,t_index]) % t_list[t_index]
plain_output = temp
temp = None
np.save("./output_data/plain_layer_5", plain_output)

Computing layer 5/5...


# INFERE ENC

In [27]:
import numpy as np
from wrapper import SEAL

In [28]:
SEALobj = SEAL()
q_list = SEALobj.q_list
k_list = SEALobj.k_list
n_parm = SEALobj.n_parm
enc_poly_size = SEALobj.enc_poly_size
q_size = len(q_list)
t_size = len(SEALobj.t_list)

In [29]:
def to_object_dtype(tensoreuint):
	shape = tensoreuint.shape
	tensoreuint.shape = (tensoreuint.size,)
	new = np.empty((tensoreuint.size,), dtype=object)
	for i in range(tensoreuint.size):
		new[i] = int(tensoreuint[i].item())
	new.shape = shape
	return new

In [30]:
# WEIGHTS
dense1_kernel = np.load("./nn_data/dense1_kernel.npy")
dense1_bias = np.load("./nn_data/dense1_bias.npy")
dense2_kernel = np.load("./nn_data/dense2_kernel.npy")
dense2_bias = np.load("./nn_data/dense2_bias.npy")
dense3_kernel = np.load("./nn_data/dense3_kernel.npy")
dense3_bias = np.load("./nn_data/dense3_bias.npy")


In [31]:
# INPUT AND OUTPUT DATA
print("Encrypting the input...")
encrypted_input = np.load("./output_data/plain_layer_0.npy") # not yet encrypted
examples_count = encrypted_input.shape[0]
encrypted_input = SEALobj.encrypt_tensor(encrypted_input) # now it is
np.save("./output_data/enc_layer_0", encrypted_input)
poly_groups_count = encrypted_input.shape[0]//enc_poly_size
encrypted_output = np.empty((encrypted_input.shape[0],22,t_size), dtype=np.uint64)



Encrypting the input...


In [32]:
# LAYER 1: fully connected layer
print("Computing layer 1/5...")
encrypted_output = to_object_dtype(encrypted_input)
dense1_kernel = to_object_dtype(dense1_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],20,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense1_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense1_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_1", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 1/5...


In [33]:

# LAYER 2: square activation function
print("Computing layer 2/5...")
encrypted_output = SEALobj.square_tensor(encrypted_output)
np.save("./output_data/enc_layer_2", encrypted_output)



Computing layer 2/5...


In [34]:
# LAYER 3: fully connected layer
print("Computing layer 3/5...")
encrypted_output = to_object_dtype(encrypted_output)
dense2_kernel = to_object_dtype(dense2_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],10,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense2_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense2_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_3", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 3/5...


In [35]:
# LAYER 4: square activation function
print("Computing layer 4/5...")
encrypted_output = SEALobj.square_tensor(encrypted_output)
np.save("./output_data/enc_layer_4", encrypted_output)


Computing layer 4/5...


In [36]:
# LAYER 5: fully connected layer
print("Computing layer 5/5...")
encrypted_output = to_object_dtype(encrypted_output)
dense3_kernel = to_object_dtype(dense3_kernel)
## kernel
temp = np.empty((encrypted_output.shape[0],2,t_size), dtype=object)
for t_index in range(t_size):
	temp[...,t_index] = encrypted_output[...,t_index].dot(dense3_kernel[...,t_index])
encrypted_output = temp
temp = None
## % q
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for size_index in range(2):
				for q_index in range(q_size):
					for n_index in range(n_parm+1):
						axis0 = poly_group_index*enc_poly_size + size_index*q_size*(n_parm+1) + q_index*(n_parm+1) + n_index
						temp = encrypted_output[axis0,axis1,axis2]
						temp = temp % q_list[q_index]
						encrypted_output[axis0,axis1,axis2] = temp
## bias
for axis1 in range(encrypted_output.shape[1]):
	for axis2 in range(encrypted_output.shape[2]):
		for poly_group_index in range(poly_groups_count):
			for q_index in range(q_size):
				axis0 = poly_group_index*enc_poly_size + ((n_parm+1)*q_index)
				temp = encrypted_output[axis0,axis1,axis2]
				temp = temp + dense3_bias[axis1,axis2].item()*k_list[axis2][q_index]
				temp = temp % q_list[q_index]
				encrypted_output[axis0,axis1,axis2] = temp
np.save("./output_data/enc_layer_5", encrypted_output) # !!!! encrypted_output has dtype=object. To load it use the function at the end of the file !!!!


Computing layer 5/5...


In [37]:
# DECRYPT
print("Decrypting the output...")
decrypted_output = SEALobj.decrypt_tensor(encrypted_output, examples_count)
encrypted_output = None
np.save("./output_data/decrypted_layer_5", decrypted_output)



Decrypting the output...


# POST_DECODE

In [38]:
import os.path

In [39]:
plain_output_exists = os.path.isfile("./output_data/plain_layer_5.npy")
decrypted_output_exists = os.path.isfile("./output_data/decrypted_layer_5.npy")

In [40]:
def extended_Euclidean_algorithm(a, b):
	b0 = b
	x0, x1 = 0, 1
	if b == 1: return 1
	while a > 1:
		q = a // b
		a, b = b, a%b
		x0, x1 = x1 - q * x0, x0
	if x1 < 0: x1 += b0
	return x1


In [41]:
def chinese_remainder_theorem(array):
	result = 0
	for t_index in range(len(array)):
		result += array[t_index].item() * bezout_coefficients[t_index] * t_product_over_t[t_index]
	return result % t_product

In [42]:
def crt_inverse(tensor):
	examples_count = tensor.shape[0]
	temp = np.empty(tensor.shape[:-1], dtype=object)
	for i in range(examples_count):
		for j in range(2):
			temp[i, j] = chinese_remainder_theorem(plain_output[i, j, :])
			if (temp[i, j]>negative_threshold):
				temp[i, j] = temp[i, j] - t_product
	return temp


In [43]:
# CRT PARAMETERS
# compute the producte of all t, and the threshold for negative numbers:
#   t_product
#   negative_threshold
t_product = 1
for t_index in range(len(t_list)):
	t_product = t_product * t_list[t_index]
negative_threshold = t_product // 2
# compute t_product // t and the Bezout coefficients, for all t: 
#   t_product_over_t
#   bezout_coefficients
t_product_over_t = []
bezout_coefficients = []
for t_index in range(len(t_list)):
	t_product_over_t.append(t_product // t_list[t_index])
	temp = extended_Euclidean_algorithm(t_product_over_t[t_index], t_list[t_index])
	bezout_coefficients.append(temp)



In [44]:
# COMPUTE PREDICTIONS
string = ""
if (plain_output_exists):
	plain_output = np.load("./output_data/plain_layer_5.npy")
if (decrypted_output_exists):
	decrypted_output = np.load("./output_data/decrypted_layer_5.npy")
	if (plain_output_exists):
		if (np.array_equal(plain_output, decrypted_output)):
			print("---- Plain and decrypted outputs coincide ----")
			cn_predictions = crt_inverse(decrypted_output)
			string = "Accuracy with SEAL encryption:"
		else:
			print("---- Plain and decrypted outputs are different. Computing accuracy with plain output ----")
			cn_predictions = crt_inverse(plain_output)
			string = "Accuracy with integer numbers (no encryption):"
	else:
		print("---- plain_layer_5.npy file is missing. Can't compare decrypted and plain outputs ----")
		cn_predictions = crt_inverse(decrypted_output)
		string = "Accuracy with SEAL encryption:"
else:
	print("---- decrypted_layer_5.npy file is missing. Can't compare decrypted and plain outputs ----")
	print("Computing accuracy with plain output...")
	cn_predictions = crt_inverse(plain_output)
	string = "Accuracy with integer numbers (no encryption):"
cn_predictions = np.argmax(cn_predictions, axis=1)


---- Plain and decrypted outputs coincide ----


In [45]:
saver = tf.train.Saver()

# PRINT ACCURACIES
with tf.Session() as sess:
	saver.restore(sess, './nn_data/net-1000')

	tf_guessed_predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
	tf_accuracy = tf.reduce_mean(tf.cast(tf_guessed_predictions, "float"))
	cn_guessed_predictions = tf.equal(cn_predictions, tf.argmax(y, 1))
	cn_accuracy = tf.reduce_mean(tf.cast(cn_guessed_predictions, "float"))


	tf_guessed_predictions_ = tf_guessed_predictions.eval({X: X_test, y: y_test})
	cn_guessed_predictions_ = cn_guessed_predictions.eval({y: y_test})
	swapped_predictions_count = 0
	for i in range(tf_guessed_predictions_.size):
		if (tf_guessed_predictions_[i]!=cn_guessed_predictions_[i]):
			swapped_predictions_count = swapped_predictions_count + 1


	print("Accuracy with tensorflow and no CRT:", tf_accuracy.eval({X: X_test, y: y_test}))
	print(string, cn_accuracy.eval({X: X_test, y: y_test}))
	print("Number of swapped predictions: ", swapped_predictions_count)

INFO:tensorflow:Restoring parameters from ./nn_data/net-1000
Accuracy with tensorflow and no CRT: 0.9183673
Accuracy with SEAL encryption: 0.9183673
Number of swapped predictions:  2
